假设我们想对函数  
$$y = 2\mathbf{x}^T\mathbf{x}$$
关于列向量x求导

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
#在我们计算y关于x的梯度之前，我们需要一个地方来储存梯度
#下面的这个代码等价于 x = torch.arange(4.0,requires_grad=True)
x.requires_grad_(True)
x.grad #默认值是None

In [3]:
#现在让我们计算y
y = 2 * torch.dot(x,x)
#dot(x,x)=14
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

1.对每个x_i求偏导数：
∂y/∂x₀ = ∂/∂x₀(2 * (x₀² + x₁² + x₂² + x₃²)) = 2 * 2x₀ = 4x₀ = 4*0 = 0
∂y/∂x₁ = ∂/∂x₁(2 * (x₀² + x₁² + x₂² + x₃²)) = 2 * 2x₁ = 4x₁ = 4*1 = 4  
∂y/∂x₂ = ∂/∂x₂(2 * (x₀² + x₁² + x₂² + x₃²)) = 2 * 2x₂ = 4x₂ = 4*2 = 8
∂y/∂x₃ = ∂/∂x₃(2 * (x₀² + x₁² + x₂² + x₃²)) = 2 * 2x₃ = 4x₃ = 4*3 = 12

2.对于 y = 2x^T x，梯度公式是：∂y/∂x = 4x
1) 首先理解 $\mathbf{x}^T \mathbf{x}$ 是什么

对于向量 $\mathbf{x} = [x_1, x_2, x_3, x_4]^T$：

$$\mathbf{x}^T \mathbf{x} = x_1^2 + x_2^2 + x_3^2 + x_4^2$$

2)  展开函数 $y = 2\mathbf{x}^T\mathbf{x}$

$$y = 2(x_1^2 + x_2^2 + x_3^2 + x_4^2)$$

3) 求偏导数（逐个分量）

$$\frac{\partial y}{\partial x_1} = \frac{\partial}{\partial x_1}[2(x_1^2 + x_2^2 + x_3^2 + x_4^2)] = 2 \cdot 2x_1 = 4x_1$$

$$\frac{\partial y}{\partial x_2} = \frac{\partial}{\partial x_2}[2(x_1^2 + x_2^2 + x_3^2 + x_4^2)] = 2 \cdot 2x_2 = 4x_2$$

$$\frac{\partial y}{\partial x_3} = 4x_3$$

$$\frac{\partial y}{\partial x_4} = 4x_4$$

4) 组合成梯度向量

$$\frac{\partial y}{\partial \mathbf{x}} = \begin{bmatrix} 4x_1 \\ 4x_2 \\ 4x_3 \\ 4x_4 \end{bmatrix} = 4 \begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{bmatrix} = 4\mathbf{x}$$

验证：4 * [0., 1., 2., 3.] = [0., 4., 8., 12.] ✓

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

现在让我们计算x的另一个函数

y = x.sum() = x₁ + x₂ + x₃ + x₄ = 0 + 1 + 2 + 3 = 6
$$y = x_1 + x_2 + x_3 + x_4$$

$$\frac{\partial y}{\partial x_1} = \frac{\partial}{\partial x_1}(x_1 + x_2 + x_3 + x_4) = 1$$

$$\frac{\partial y}{\partial x_2} = \frac{\partial}{\partial x_2}(x_1 + x_2 + x_3 + x_4) = 1$$

$$\frac{\partial y}{\partial x_3} = 1$$

$$\frac{\partial y}{\partial x_4} = 1$$

In [6]:
#在默认情况下，PyTorch会积累梯度，需要我们清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

在深度学习中，我们的目的不是计算微积分矩阵，而是批量中每个样本单独计算的偏导数之和

1. y = x * x 计算什么？
对于 x = [0., 1., 2., 3.]：
y = x * x = [0², 1², 2², 3²] = [0., 1., 4., 9.]
这里 y 是一个向量（Vector），不是标量（Scalar）
在PyTorch中：
向量 = 一维数组（多个数字排成一排）
标量 = 单个数字
方向不是重点，形状才是重点
2. y.sum() 是什么？
y.sum() = 0 + 1 + 4 + 9 = 14
所以实际求导的函数是：
f = y.sum() = x₁² + x₂² + x₃² + x₄²

$$f = x_1^2 + x_2^2 + x_3^2 + x_4^2$$

$$\frac{\partial f}{\partial x_1} = \frac{\partial}{\partial x_1}(x_1^2 + x_2^2 + x_3^2 + x_4^2) = 2x_1$$

$$\frac{\partial f}{\partial x_2} = \frac{\partial}{\partial x_2}(x_1^2 + x_2^2 + x_3^2 + x_4^2) = 2x_2$$

$$\frac{\partial f}{\partial x_3} = 2x_3$$

$$\frac{\partial f}{\partial x_4} = 2x_4$$

$$\frac{\partial f}{\partial \mathbf{x}} = 2\mathbf{x} = 2[x_1, x_2, x_3, x_4]^T$$

In [7]:
#对非标量调用backward需要传入一个gradient参数，该参数指定微积分函数
x.grad.zero_()
y = x * x
#等价于y.backward(torch.ones(len(x)))
#梯度 = 2x = 2[0., 1., 2., 3.] = [0., 2., 4., 6.]
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [8]:
#把y当成常数而不是函数赋值给u
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构造函数的计算图需要通过Python控制流(例如：条件，循环或任意函数调用)，我们仍然可以计算得到变量的梯度

In [10]:
def f(a):
    b = a * 2                    # b = 2a
    while b.norm() < 1000:       # 当|b| < 1000时继续循环
        b = b * 2                # b = b * 2
    if b.sum() > 0:              # 如果b > 0
        c = b                    # c = b
    else:                        # 如果b ≤ 0  
        c = 100 * b              # c = 100b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)